In [1]:
!pip install icecream

In [51]:
from io import StringIO #input output library  -> used for extracting
import requests #Web scrapping used for communicating with web pages (get hit and upload operations) -> used for extract
from bs4 import BeautifulSoup #used for parsing HTML code of webpage
import pandas as pd # used ofr transformation
import sqlite3 #loading the data in small database (small version of SQL)
from datetime import datetime
#from icecream import ic #debugging visualization

In [52]:
#Step :0: to observe the logs to know when error occur during the program failure at what step(i.e extraction etc)

def log_progress(message):
    with open("./project_1_file", 'a') as file:
        file.write(f'{datetime.now()},{message}./n')

In [53]:
#step 1 Extraction:

def extraction(url1, table_attribs):
    soup = requests.get(url1).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span',  string=table_attribs).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]
    
    log_progress('Data extraction complete. Initiating Transformation process')

    return df

In [54]:
#step:2: transform:

def transformation(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col = 0).to_dict()['Rate']
    
    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)']* exchange_rate['GBP'], 2) #2 defines that there will be 2 vals after the dec point
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)']* exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)']* exchange_rate['INR'], 2)
    
    print(df['MC_EUR_Billion'][4])
    
    log_progress('Data transformation complete. Initiating Loading process')

    return df

In [55]:
#step:3: Load
#csv

def load_to_csv(df, out_path):
    df.to_csv(out_path)
    
    log_progress('Data saved to CSV file')

In [56]:
#step:3:
#SQLite

def load_to_db(df, sql_conn, tn): #dataframe , connection, table name
    df.to_sql(tn, sql_conn, if_exists = 'replace', index = False)
    
    log_progress('Data loaded to Database as a table, Executing queries')

In [57]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    #
    log_progress('Preliminaries complete. Initiating ETL process')
    #
    df= extraction(url, 'By market capitalization')
    log_progress('extraction completed')
    print(df)
    
    
    df = transformation(df , 'exchange_rate.csv')
    log_progress('Extraction Completed')
    print(df)
    
    load_to_csv(df, output_csv_path)
    log_progress('DataFrame has been uploaded into the csv file on our system')
    
    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)
    log_progress("DataFrame has been uploaded into the SQLite Database succesfully!")

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81
146.86
   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2     